# Extract dwell-times from fast5 base-directories

Chris Kimmel, 8/6/2020 (kimmel.95@osu.edu)

This code will extract dwell-times from Tombo fast5 files.
I will use the Tombo API.
After extracting dwell-times to CSV, I will perform a sanity check by comparing my results to results from previous attempts.

Each output CSV from this notebook will correspond to a single fast5 base-directory, and will be structured as follows.
> Each entry in the CSV file will be an integer representing the dwell-time of a particular read at a particular genomic position.
Dwell-times are integers, not floats, because they are measured in number of raw current measurements assigned to the corresponding base.
(Dwell-times are not measured in milliseconds.)
>
> The rows of the CSV file will be labeled with read ids. 
The columns will be labeled with zero-based positional indexes into the sequence of the reference fasta that was used for resquiggling.
> 
> Fasta files are written in the 5'->3' direction, and the positional indexes will increase in this direction as well.
(Index 0 corresponds to the furthest nucleotide on the 5' end of the reference fasta sequence.)

Familiarity with the Tombo Python API is assumed throughout this notebook, but that API isn't well-documented.
Please email me if you're reading my code and you want me to explain or clarify anything.

---

### Data source:

The input data is imported from filepaths on the Ohio Supercomputer filesystem.
It consists of directories of single-fast5 files.
Each directory is accompanied by a Tombo index file.

In [132]:
import os.path
from collections import OrderedDict # Strictly for style, not functionality
import datetime

import numpy as np
import pandas as pd
from tombo import tombo_helper

In [14]:
fast5_basedir_path_list = [
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_A/Trizol_A_8K_single_fast5',
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_B/Trizol_B_8K_single_fast5',
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_C/Trizol_C_8K_single_fast5',
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_OLD/Trizol_OLD_8K_single_fast5',
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_GL',
    '/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_subsampled',
]

reference_fasta_path = '/fs/project/PAS1405/General/Kimmel_Chris/RNA_section__454_9627.fa'

chromosome, strand = 'truncated_hiv_rna_genome', '+'
corrected_group = 'RawGenomeCorrected_000'
basecall_subgroup = 'BaseCalled_template'

In [43]:
fast5_basedir_path_dict = OrderedDict((os.path.basename(path), path)
                                      for path in fast5_basedir_path_list)
fast5_basedir_path_dict

OrderedDict([('Trizol_A_8K_single_fast5',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_A/Trizol_A_8K_single_fast5'),
             ('Trizol_B_8K_single_fast5',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_B/Trizol_B_8K_single_fast5'),
             ('Trizol_C_8K_single_fast5',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_C/Trizol_C_8K_single_fast5'),
             ('Trizol_OLD_8K_single_fast5',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_OLD/Trizol_OLD_8K_single_fast5'),
             ('f1f2_GL',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_GL'),
             ('f1f2_subsampled',
              '/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_subsampled')])

## Data prep

In [45]:
# Open all the .tombo.index files

index_dict = OrderedDict()
for name, path in fast5_basedir_path_dict.items():
    print(name)
    index_dict[name] = tombo_helper.TomboReads(
        fast5s_basedirs=[path],
        corrected_group=corrected_group,
        basecall_subgroups=[basecall_subgroup],
        # Default is basecall_subgroups=None, which makes Tombo include reads
        # from all basecall subgroups in the fast5 file. It shouldn't matter.
        for_writing=False,
        remove_filtered=True,
        # Unless told otherwise, Tombo will filter out reads with low
        # sequence-to-signal alignment. This is recorded in the index file,
        # so we use remove_filtered=True.
        sample_name=name # just for verbose output
    )
    print('')

index_dict

[14:48:20] Parsing Trizol_A_8K_single_fast5 Tombo index file(s).
[14:48:20] Parsing Trizol_B_8K_single_fast5 Tombo index file(s).
[14:48:20] Parsing Trizol_C_8K_single_fast5 Tombo index file(s).
[14:48:20] Parsing Trizol_OLD_8K_single_fast5 Tombo index file(s).
[14:48:20] Parsing f1f2_GL Tombo index file(s).


Trizol_A_8K_single_fast5

Trizol_B_8K_single_fast5

Trizol_C_8K_single_fast5

Trizol_OLD_8K_single_fast5

f1f2_GL


[14:48:23] Parsing f1f2_subsampled Tombo index file(s).



f1f2_subsampled



OrderedDict([('Trizol_A_8K_single_fast5',
              <tombo.tombo_helper.TomboReads at 0x2b1a46c28d90>),
             ('Trizol_B_8K_single_fast5',
              <tombo.tombo_helper.TomboReads at 0x2b1a46475610>),
             ('Trizol_C_8K_single_fast5',
              <tombo.tombo_helper.TomboReads at 0x2b1a46bc1550>),
             ('Trizol_OLD_8K_single_fast5',
              <tombo.tombo_helper.TomboReads at 0x2b1a46c28a50>),
             ('f1f2_GL', <tombo.tombo_helper.TomboReads at 0x2b1a46c283d0>),
             ('f1f2_subsampled',
              <tombo.tombo_helper.TomboReads at 0x2b1a46475e50>)])

It looks like Tombo might not have found index files for some of the reads.
Let's just check that they're there.

In [44]:
for name, path in fast5_basedir_path_dict.items():
    index_basename = '.' + name + '.' + corrected_group + '.tombo.index'
    index_path = os.path.join(os.path.dirname(path), index_basename)
    if os.path.exists(index_path):
        print('The index file exists for ' + path)
    else:
        print('The index file does not exist for ' + path)

The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_A/Trizol_A_8K_single_fast5
The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_B/Trizol_B_8K_single_fast5
The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_C/Trizol_C_8K_single_fast5
The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/sequencing_data/Trizol_OLD/Trizol_OLD_8K_single_fast5
The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_GL
The index file exists for /fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_subsampled


Yes, all the Tombo index files exist already.

As long as we're checking things, lets check our assumption that every tombo index file contains reads only for the expected chromosome and strand.

In [48]:
for name, index in index_dict.items():
    print(name)
    print(index.get_all_cs())
    print('')

Trizol_A_8K_single_fast5
[(u'truncated_hiv_rna_genome', u'+')]

Trizol_B_8K_single_fast5
[(u'truncated_hiv_rna_genome', u'+')]

Trizol_C_8K_single_fast5
[(u'truncated_hiv_rna_genome', u'+')]

Trizol_OLD_8K_single_fast5
[(u'truncated_hiv_rna_genome', u'+')]

f1f2_GL
[(u'truncated_hiv_rna_genome', u'+')]

f1f2_subsampled
[(u'truncated_hiv_rna_genome', u'-'), (u'truncated_hiv_rna_genome', u'+')]



That's disturbing. The `f1f2_subsampled` fast5 base-directory has reads for the reverse strand.

Let's investigate.

In [50]:
weird_reads = index_dict['f1f2_subsampled'].get_cs_reads(chromosome, '-')
print(len(weird_reads))
weird_reads

1


[readData(start=3, end=3872, filtered=False, read_start_rel_to_raw=417, strand=u'-', fn=u'/fs/project/PAS1405/GabbyLee/project/m6A_modif/ctrl_data_set/data_Olivier/f1f2_subsampled/f385ffd1-699e-41fe-9c02-da2a8bc8a0c6.fast5', corr_group=u'RawGenomeCorrected_000/BaseCalled_template', rna=True, sig_match_score=1.1738530947767862, mean_q_score=13.772344013490725, read_id=u'f385ffd1-699e-41fe-9c02-da2a8bc8a0c6')]

Evidently that was the only read on the reverse strand.
It's surprising that the read had a `sig_match_score` of 1.17;
that's not for `tombo resquiggle` to filter it out (under the default options at least).

Maybe this molecule just went through the nanopore backwards.
But then why doesn't this read show up on some other dataset?

I'm letting it go and moving on.

## Extract dwell-times
#### (...and genomic base assignments)

The point of this notebook is to extract dwell-times.
We're also extracting genomic base assignments so that we can make sure our dwell-time data lines up perfectly with our reference genome.

This section of the notebook assumes familiarity with fast5 filesin general, and fast5 "Events" datasets in particular.

---

#### Digression on the "Events" table

Inside the fast5 file, where the resquiggling results are stored, is an HDF5 dataset called "Events".
This dataset is a table with several columns.
There is one row in the Events table for every position on the reference genome sequence (i.e., the sequence from the reference fasta).

One column, labeled 'base', is filled with single A's, C's, T's, and G's.
These match the bases in the corresponding positions of the reference fasta.
Another column, labeled 'length', gives the number of raw current measurements attributed to each nucleotide on the reference fasta.
The length is, essentially, the dwell-time.

There is no 'position' column, unfortunately, but it is easy to infer.
To compute the zero-based reference-genome nucleotide position of a row in the table, use this rule:
> The position of the first row of the Events table is given by
the `start` attribute of the tombo `readData` object corresponding to the read.
The next row of the Events table is the next position on the reference genome, and this pattern continues until the end of the read.
>
> The total number of rows in the Events table equals `readData.end - readData.start`.

---

Here's a function to extract columns of the Events table as pandas Series.

In [97]:
def extract_column_as_series(read, column_name):
    '''Extract a column from the Events table of this read's fast5 file.
    
    Args:
        read (tombo.tombo_helper.readData): A Tombo readData object. The Events
            dataset in the corresponding fast5 file will be opened, and the
            specified column extracted.
        column_name (str): The name of the column to extract from the Events
            table. Options are: 'norm_mean', 'norm_stdev', 'start', 'length',
            and 'base'.
    
    Returns:
        pandas.Series: A pandas series filled with event lengths
            
            Name: The name of this series will be the read id.
            
            Index: The index labels will describe positions along the read.
            Specifically, the index label corresponding to position i on the
            reference genome will be labeled with the integer i.
            
            Like Tombo, we use zero-based indexing into the reference genome
            sequence. That is, what we call "position 0" corresponds to the
            nucleotide at the beginning of the reference FASTA.
    '''
    column = tombo_helper.get_single_slot_genome_centric(read, column_name)
    # get_single_slot_genome_centric reverses '-' strands, but that doesn't
    # matter for us since we only have '+' strands
    
    events_based_index = np.arange(len(column))
    reference_based_index = events_based_index + read.start
    
    return pd.Series(
        column,
        name=read.read_id,
        index=reference_based_index
    )

# quick test
extract_column_as_series(
    index_dict['f1f2_GL'].get_cs_reads(chromosome, strand)[0],
    'base'
).head()

4597    G
4598    A
4599    T
4600    G
4601    G
Name: 7eea708e-b3e2-4bda-b986-d2611ae50e5c, dtype: object

We have function that can extract a column from an individual read.
Let's make a function that can extract a column from all the reads in a `readData` object and collate them into a single table.

In [124]:
def extract_column_from_all(list_of_reads, column_name):
    '''Extract `column_name` from events tables of reads in `list_of_reads`
    
    Args:
        list_of_reads (`list` of `readData`): A list of reads. It is intended
            they are all of the same chromosome and strand, but this cannot
            be checked.
    
    Returns:
        pandas.DataFrame: Dataframe filled with entries from the column
            named `column_name` in the events tables of the reads in
            `list_of_reads`.
            
            Rows are labeled with read ids. Columns are labeled with zero-based
            nucleotide-positions relative to the start of the reference fasta
            genome.
    '''
    
    return pd.concat(
        [extract_column_as_series(read, column_name)
         for read in list_of_reads],
        axis=1,
        join='outer', # default
        ignore_index=False, # default
    ).T # note the transpose


# quick test
observed = extract_column_from_all(
    index_dict['f1f2_GL'].get_cs_reads(chromosome, strand)[0:200],
    'base'
)
assert all([np.array_equal(x, y) # in python 3 add kwarg `equal_nan=True`
            for x, y in zip(observed.T, observed.T.iloc[1:,:])])
print('test successful')


observed

test successful


,1,2,3,4,5,6,7,8,9,10,...,9161,9162,9163,9164,9165,9166,9167,9168,9169,9170
7eea708e-b3e2-4bda-b986-d2611ae50e5c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,G,A,G,T,G,C,T,T
3321a304-f6b7-46de-a9c7-6e21228ea24e,NaN,NaN,NaN,NaN,NaN,NaN,T,C,T,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4774288a-63ba-46f3-83db-9782036bce6a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3bbc2256-2c2f-4fa6-8d8d-465d88ef3cf6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,G,A,G,T,G,C,T,NaN
b8519155-6299-4653-b4a2-f9b3d3b9f6bb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,G,A,G,T,G,C,T,T
102b6260-e49a-4293-a0cd-98950a6d3741,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,G,A,G,T,G,C,T,NaN
d2d66b3c-fa26-4be3-889c-718c912f8848,NaN,NaN,NaN,NaN,T,C,T,C,T,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f8b1981e-e1b1-4376-b2d7-f52ef7a771f3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,T,T,G,A,G,T,G,C,T,NaN
9d5514f5-3b69-414c-a64d-7efc58810d8c,NaN,NaN,NaN,NaN,T,C,T,C,T,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d0eee1f5-3409-434f-af36-000725ec3c9d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we have enough tools to extract the desired table of dwell times from each fast5 base-directory.

In [125]:
# took about two hours (Owens cluster, 1 node, 28 cores, plain vanilla filestystem)
# to cut the processing time in half, comment out the base_table lines

length_table_dict = OrderedDict()
base_table_dict = OrderedDict()
for name, index in index_dict.items():
    print(name)
    list_of_reads = index.get_cs_reads(chromosome, strand)
    print('extracting length table...')
    length_table_dict[name] = extract_column_from_all(list_of_reads, 'length')
    print('extracting base table...')
    base_table_dict[name] = extract_column_from_all(list_of_reads, 'base')
    print('')
print('done')

Trizol_A_8K_single_fast5
extracting length table...
extracting base table...

Trizol_B_8K_single_fast5
extracting length table...
extracting base table...

Trizol_C_8K_single_fast5
extracting length table...
extracting base table...

Trizol_OLD_8K_single_fast5
extracting length table...
extracting base table...

f1f2_GL
extracting length table...
extracting base table...

f1f2_subsampled
extracting length table...
extracting base table...

done


Now it's time to output the length tables to CSV files.

In [127]:
for name, length_table in length_table_dict.items():
    print(name)
    output_filename = name + '_dwells.csv'
    length_table.to_csv(
        output_filename,
        sep=',', # default
        na_rep='nan',
        header=True,
        index=True,
        index_label='read_id'
    )
    print('')
print('done')

Trizol_A_8K_single_fast5

Trizol_B_8K_single_fast5

Trizol_C_8K_single_fast5

Trizol_OLD_8K_single_fast5

f1f2_GL

f1f2_subsampled

done


The above code seems to print CSV files full of entries that look like "12.0" instead of "12".
You could cut down the size of the CSV files considerably if you fixed that...

## Sanity checks

### Reference FASTA

Let's check the base tables against the reference genome to make sure the position-numbers are correct.

In [130]:
# Get reference sequence as a pandas Series

with open(reference_fasta_path, 'rt') as file:
    lines = file.readlines()
    
reference_sequence = lines[1].strip().upper()
reference_sequence_series = pd.Series(
    list(reference_sequence),
    index=np.arange(len(reference_sequence))
)

reference_sequence_series

0       G
1       G
2       G
3       T
4       C
5       T
6       C
7       T
8       C
9       T
10      G
11      G
12      T
13      T
14      A
15      G
16      A
17      C
18      C
19      A
20      G
21      A
22      T
23      C
24      T
25      G
26      A
27      G
28      C
29      C
       ..
9144    C
9145    C
9146    T
9147    C
9148    A
9149    A
9150    T
9151    A
9152    A
9153    A
9154    G
9155    C
9156    T
9157    T
9158    G
9159    C
9160    C
9161    T
9162    T
9163    G
9164    A
9165    G
9166    T
9167    G
9168    C
9169    T
9170    T
9171    C
9172    A
9173    A
Length: 9174, dtype: object

In [228]:
# Compare each row of base_table to the reference series
# Admittedly, this probably uses more computation than is really necessary

# This cell did not finish, even after three hours (Owens Cluster, 1 node, 28 cores)

for name, base_table in base_table_dict.items():
    print(name)
    for index, row in base_table.iterrows():
        # combine dataframes so they have the same index
        df = pd.DataFrame([row, reference_sequence_series])
        obs_bases = df.iloc[0, :].values.astype('U')
        ref_bases = df.iloc[1, :].values.astype('U')
        # Check that our bases equal the reference genome bases.
        # Make exceptions for bases that aren't defined.
        assert np.all((obs_bases == ref_bases)
                      | (obs_bases == 'nan').astype('?')
                     ), index
print('test passed')

Trizol_A_8K_single_fast5
Trizol_B_8K_single_fast5
Trizol_C_8K_single_fast5
Trizol_OLD_8K_single_fast5
f1f2_GL


KeyboardInterrupt: 

I interrupted execute of the preceeding cell after about three hours.
It verified that the bases matched the reference genome for all these samples:
* Trizol_A_8K_single_fast5
* Trizol_B_8K_single_fast5
* Trizol_C_8K_single_fast5
* Trizol_OLD_8K_single_fast5

Unfortunately f1f2_GL, with over 100,000 samples, was just taking too dang long.

### Previous work

Let's make sure our new tables agree with similar tables produced by earlier code.

Two things need to be noted:
1. Earlier code did not filter out reads with poor sequence-to-signal alignment, so earlier CSV files will include reads that aren't in our new tables.
2. Some (but not all) earlier datasets had rows labeled like "3154b6e-f9cc-4e3b-8c2c-5f0fa43de9c2.fast5", but we label them like "13154b6e-f9cc-4e3b-8c2c-5f0fa43de9c2" (without the .fast5 extension).

I will just load one CSV produced from earlier code.
The complexity of checking several isn't worth it.

In [271]:
earlier_csv_path = '/users/PAS1405/kimmel/extract_dwells_2/Trizol_B_lengths.csv'
corresponding_name = 'Trizol_B_8K_single_fast5'

earlier_csv_dataframe = pd.read_csv(
    earlier_csv_path,
    header=0,
    index_col=0
)

# Correct the incorrect index labels
earlier_csv_dataframe.index = earlier_csv_dataframe.index.map(
    lambda x: x.split('.')[0]
)
earlier_csv_dataframe.columns = earlier_csv_dataframe.columns.astype(np.dtype(int))

# Check that new reads agree with old reads
for read_id, row in length_table_dict[corresponding_name].iterrows():
    earlier_row = earlier_csv_dataframe.loc[read_id, :]
    assert all(
        (earlier_row == row)
        | (earlier_row.isna() & row.isna())
    ), read_id

# Check that the neither table is very much bigger than the other
ratio = len(earlier_csv_dataframe)/len(length_table_dict[corresponding_name])
assert 0.8 < ratio < 1.2, ratio
print('test passed')

test passed


### Filtered reads

Let's make sure that filtered reads were not included in the output tables

In [276]:
for name in index_dict.keys():
    print(name)
    
    index = index_dict[name]
    filtered_read_ids = [read.read_id for read in index.get_cs_reads(chromosome, strand) if read.filtered]
    unfiltered_read_ids = [read.read_id for read in index.get_cs_reads(chromosome, strand) if not read.filtered]
    
    # check that NONE of the FILTERED reads are represented in the index of the dwell-time table
    length_table_index_set = set(length_table_dict[name].index)
    if all([read_id not in length_table_index_set for read_id in filtered_read_ids]):
        pass
    else:
        print('test 1 failed: ' + name)
    
    # check that ALL of the UNFILTERED read ids are represented in the index of the dwell-time table
    if all([read_id in length_table_index_set for read_id in unfiltered_read_ids]):
        pass
    else:
        print('test 2 failed: ' + name)

Trizol_A_8K_single_fast5
Trizol_B_8K_single_fast5
Trizol_C_8K_single_fast5
Trizol_OLD_8K_single_fast5
f1f2_GL
f1f2_subsampled


This test also passed. So the dwell-times passed all our tests, and we'll consider them correct.

## KS Tests

This is not related to anything else in the notebook; I just wanted to try it.
This is a positional two-sample KS test of Trizol A reads against f1f2 reads.

In [286]:
pos = 1234
sample_A = length_table_dict['Trizol_A_8K_single_fast5'].loc[:, pos].dropna()
sample_B = length_table_dict['f1f2_subsampled'].loc[:, pos].dropna()

print(len(sample_A))
print(len(sample_B))

import scipy.stats as stats
stats.ks_2samp(sample_A, sample_B)

392
3441


Ks_2sampResult(statistic=0.12666806042382084, pvalue=2.141212982034808e-05)